# Gevent

To install:

    $ sudo apt-get install python-dev libevent-dev
    $ virtualenv gevent
    $ source ~/gevent/bin/activate
    (gevent)$ pip install gevent

To use the notebook:

    (gevent)$ sudo apt-get install libzmq-dev
    (gevent)$ pip install tornado pyzmq ipython
    (gevent)$ ipython notebook

In [2]:
!pip install gevent

  Using cached gevent-1.1.1-cp27-cp27m-macosx_10_6_intel.whl
    100% |████████████████████████████████| 81kB 2.2MB/s 
  Running setup.py bdist_wheel for greenlet ... - \ | done
  Stored in directory: /Users/rick446/Library/Caches/pip/wheels/54/85/ff/b9794e7f8dd8b845a7712885d2f7e8513d0e8fc1c91e01bc6f
Successfully built greenlet


In [66]:
import gevent

## Creating a greenlet

In [67]:
def gl_target(a, b, c):
    print a,b,c
    return 42

The easiest way is through the `spawn*` helpers

In [5]:
gl = gevent.spawn(gl_target, 1, 2, c=3)
gl

<Greenlet at 0x1065bf690: gl_target(1, 2, c=3)>

In [6]:
gl.join()
gl.value

1 2 3


42

Sometimes you want to be notified when the greenlet completes. If you use `spawn_link`, you will receive an exception whenever the greenlet terminates:

In [17]:
gl = gevent.spawn(gl_target, 1, 2, c=3)
def callback(g):
    print 'Callback', g
gl.link_value(callback)

In [21]:
try:
    gevent.sleep(0)
except gevent.greenlet.LinkedCompleted, err:
    print 'err', err
    gl.join()
    print 'val', gl.value

In [22]:
def gl_target_err():
    raise ValueError

In [23]:
gl = gevent.spawn_link(gl_target_err)
gl

AttributeError: 'module' object has no attribute 'spawn_link'

In [9]:
try:
    gevent.sleep(0)
except gevent.greenlet.LinkedExited, err:
    print 'Error caught:', err
    gl.join()
    print 'Greenlet joined'

Error caught: <Greenlet at 0x19fd7d0: gl_target_err> failed with ValueError
Greenlet joined


Traceback (most recent call last):
  File "/home/vagrant/bigdive/local/lib/python2.7/site-packages/gevent/greenlet.py", line 390, in run
    result = self._run(*self.args, **self.kwargs)
  File "<ipython-input-7-c006aea2c7a5>", line 2, in gl_target_err
    raise ValueError
ValueError
<Greenlet at 0x19fd7d0: gl_target_err> failed with ValueError



You can also be notified *only* on successful completion or *only* on exception:

In [10]:
gl = gevent.spawn_link_exception(gl_target, 1, 2, 3)
gl

<Greenlet at 0x19fd870: gl_target(1, 2, 3)>

In [11]:
gl.join()

1 2 3


In [12]:
gl = gevent.spawn_link_value(gl_target_err)
gl

<Greenlet at 0x19fda50: gl_target_err>

In [13]:
try:
    gl.join()
except:
    print 'Exception was raised'
else:
    print 'No exception was raised'
    print 'gl.exception is ', repr(gl.exception)

No exception was raised
gl.exception is  ValueError()


Traceback (most recent call last):
  File "/home/vagrant/bigdive/local/lib/python2.7/site-packages/gevent/greenlet.py", line 390, in run
    result = self._run(*self.args, **self.kwargs)
  File "<ipython-input-7-c006aea2c7a5>", line 2, in gl_target_err
    raise ValueError
ValueError
<Greenlet at 0x19fda50: gl_target_err> failed with ValueError



## Using the `gevent.Greenlet` class directly

In [24]:
gl = gevent.Greenlet(gl_target, 1, 2, 3)

In [25]:
gevent.sleep(0)

In [26]:
gl.start()

In [27]:
gevent.sleep(0)

1 2 3


In [28]:
gl.join()

In [29]:
gl = gevent.Greenlet(gl_target, 1, 2, 3)
gl.link()
gl.start()

TypeError: link() takes at least 2 arguments (1 given)

In [30]:
try:
    gevent.sleep(0)
except gevent.greenlet.LinkedExited, err:
    print err

## Timeouts

In [32]:
t = gevent.Timeout(2)
t.start()
try:
    gevent.sleep(2.1)
except gevent.Timeout, err:
    print type(err), err

<class 'gevent.timeout.Timeout'> 2 seconds


In [33]:
t = gevent.Timeout(2)
t.start()
try:
    gevent.sleep(1)
    t.cancel()
except gevent.Timeout, err:
    print type(err), err
print 'After timeout'

After timeout


In [34]:
try:
    with gevent.Timeout(2):
        gevent.sleep(2.1)
except gevent.Timeout, err:
    print err

2 seconds


In [35]:
try:
    with gevent.Timeout(2):
        gevent.sleep(1)
    print 'After timeout'
except gevent.Timeout, err:
    print err

After timeout


## Working with multiple greenlets

In [36]:
gls = [ gevent.spawn(gl_target, 1, 2, 3) for x in range(10) ]

In [37]:
gevent.joinall(gls)

1 2 3
1 2 3
1 2 3
1 2 3
1 2 3
1 2 3
1 2 3
1 2 3
1 2 3
1 2 3


## Killing a greenlet

In [38]:
import sys
def runs_a_long_time():
    for x in range(10):
        print x
        sys.stdout.flush()
        gevent.sleep(0.5)
    print 'Exit'

In [39]:
gl = gevent.spawn(runs_a_long_time)

In [40]:
gevent.sleep(5)

0
1
2
3
4
5
6
7
8
9


In [41]:
gl.kill()

### Exercise

- Install gevent
- Use the function below as a DNS lookup. Spawn this function to do parallel DNS lookups for the following domains:
  `['google.com', 'bing.com', 'baidu.com', 'axant.it', 'pypi.python.org' ]`

Sample DNS code:

    from gevent import socket
    
    def lookup_dns(name):
        # try 3 times to handle DNS timeouts
        for x in range(3):
            try:
                return socket.gethostbyname(name)
            except socket.gaierror, ex:
                print '%s: error %s' % (name, ex)
                continue

### Solution

In [42]:
from gevent import socket

def lookup_dns(name):
    # try 3 times to handle DNS timeouts
    for x in range(3):
        try:
            return socket.gethostbyname(name)
        except socket.gaierror, ex:
            print '%s: error %s' % (name, ex)
            continue


In [43]:
lookup_dns('www.axant.it')

'81.31.148.102'

In [44]:
names = ['google.com', 'bing.com', 'www.bigdive.eu', 'www.axant.it', 'pypi.python.org' ]
gls = [ gevent.spawn(lookup_dns, name) for name in names ]
gevent.joinall(gls)
for name, gl in zip(names, gls):
    print name, gl.value

google.com 216.58.217.174
bing.com 204.79.197.200
www.bigdive.eu 194.116.72.165
www.axant.it 81.31.148.102
pypi.python.org 199.27.76.223


## Greenlet synchronization

In [45]:
def wait_for_event(ev):
    ev.wait()
    print '%s is set' % ev
    sys.stdout.flush()

In [46]:
import gevent.event
ev = gevent.event.Event()

In [47]:
gl = gevent.spawn(wait_for_event, ev)

In [48]:
gl.join(1.0)

In [49]:
ev.set()

In [50]:
gl.join(1.0)

<Event set _links[0]> is set


In [51]:
def producer(q):
    for x in range(10):
        print 'Put %d' % x
        sys.stdout.flush()
        q.put(x)
        gevent.sleep(0)
        
def consumer(q):
    for x in q:
        print 'Get %d' % x
        

q = gevent.queue.Queue()
gls = [ gevent.spawn(producer, q), gevent.spawn(consumer, q) ]

AttributeError: 'module' object has no attribute 'queue'

In [53]:
gevent.sleep(5)

In [54]:
gevent.killall(gls)

In [55]:
def gl_worker(i):
    for j in range(3):
        print 'gl_worker %d: %d' % (i,j)
        gevent.sleep(1)

In [56]:
from gevent.coros import Semaphore
sem = Semaphore()
gls = [ gevent.spawn(gl_worker, i) 
        for i in range(3) ]
gevent.joinall(gls)

/Users/rick446/.virtualenvs/intermediate-python/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: gevent.coros has been renamed to gevent.lock
  if __name__ == '__main__':


gl_worker 0: 0
gl_worker 1: 0
gl_worker 2: 0
gl_worker 0: 1
gl_worker 1: 1
gl_worker 2: 1
gl_worker 0: 2
gl_worker 1: 2
gl_worker 2: 2


In [57]:
def gl_worker(i, sem):
    sem.acquire()
    for j in range(3):
        print 'gl_worker %d: %d' % (i,j)
        gevent.sleep(1)
    sem.release()

In [58]:
sem = Semaphore()
gls = [ gevent.spawn(gl_worker, i, sem) 
        for i in range(3) ]
gevent.joinall(gls)

gl_worker 0: 0
gl_worker 0: 1
gl_worker 0: 2
gl_worker 1: 0
gl_worker 1: 1
gl_worker 1: 2
gl_worker 2: 0
gl_worker 2: 1
gl_worker 2: 2


In [59]:
sem = Semaphore(2)
gls = [ gevent.spawn(gl_worker, i, sem) 
        for i in range(3) ]
gevent.joinall(gls)

gl_worker 0: 0
gl_worker 1: 0
gl_worker 0: 1
gl_worker 1: 1
gl_worker 0: 2
gl_worker 1: 2
gl_worker 2: 0
gl_worker 2: 1
gl_worker 2: 2


## Greenlet pools (limiting concurrency)

In [60]:
import gevent.pool
pool = gevent.pool.Pool(4)

In [61]:
def gl_worker(x):
    print 'Running worker %d' % x
    gevent.sleep(1)
    print 'Worker %d complete' % x

In [62]:
for x in range(10):
    pool.spawn(gl_worker, x)
gevent.sleep(4)

Running worker 0
Running worker 1
Running worker 2
Running worker 3
Worker 0 complete
Worker 1 complete
Worker 2 complete
Worker 3 complete
Running worker 4
Running worker 5
Running worker 6
Running worker 7
Worker 4 complete
Worker 5 complete
Worker 6 complete
Worker 7 complete
Running worker 8
Running worker 9
Worker 8 complete
Worker 9 complete


### Exercise

- Rewrite the DNS resolver above to limit concurrency to 2 using a Pool
- Rewrite the DNS resolver above to limit concurrency to 2 using a Semaphore

### Solution

The pool solution is quite simple:

In [63]:
names = ['google.com', 'bing.com', 'www.bigdive.eu', 'www.axant.it', 'pypi.python.org' ]
pool = gevent.pool.Pool(2)
gls = [ pool.spawn(lookup_dns, name) for name in names ]
gevent.joinall(gls)
for name, gl in zip(names, gls):
    print name, gl.value

google.com 216.58.217.174
bing.com 204.79.197.200
www.bigdive.eu 194.116.72.165
www.axant.it 81.31.148.102
pypi.python.org 199.27.76.223


Using a semaphore requires us to update our `lookup_dns` method to take a semaphore as its first argument:

In [64]:
def lookup_dns(name, sem):
    with sem:
        print 'Begin lookup %s' % name
        sys.stdout.flush()
        # try 3 times to handle DNS timeouts
        for x in range(3):
            try:
                result = socket.gethostbyname(name)
                print 'Lookup %s complete' % name
                sys.stdout.flush()
                return result
            except socket.gaierror, ex:
                print '%s: error %s' % (name, ex)
                sys.stdout.flush()
                continue

In [65]:
sem = Semaphore(2)
names = ['google.com', 'bing.com', 'www.bigdive.eu', 'www.axant.it', 'pypi.python.org' ]
gls = [ gevent.spawn(lookup_dns, name, sem) for name in names ]
gevent.joinall(gls)
for name, gl in zip(names, gls):
    print name, gl.value

Begin lookup google.com
Begin lookup bing.com
Lookup google.com complete
Begin lookup www.bigdive.eu
Lookup bing.com complete
Begin lookup www.axant.it
Lookup www.bigdive.eu complete
Begin lookup pypi.python.org
Lookup www.axant.it complete
Lookup pypi.python.org complete
google.com 216.58.217.174
bing.com 204.79.197.200
www.bigdive.eu 194.116.72.165
www.axant.it 81.31.148.102
pypi.python.org 199.27.76.223
